In [2]:
!pip install xgboost

In [3]:
import xgboost

In [4]:
from xgboost import XGBRegressor

In [5]:
import pandas as pd

In [6]:
dataset=pd.read_csv("insurance_pre.csv")

In [7]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [8]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [9]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [10]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [11]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]

In [12]:
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [13]:
dependent=dataset[["charges"]]

In [14]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [10, 100],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.3]}
grid = GridSearchCV(XGBRegressor(), param_grid, refit = True, verbose= 3,n_jobs=-1)
# fitting the model for grid search
#grid.fit(X_train, y_train)
grid.fit(independent, dependent)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    lear...=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.3],
                         'max_depth': [3, 6, 10], 'n_estimators': [10, 100]},
             verbose=3)

In [15]:
re=grid.cv_results_
grid_predictions = grid.predict(independent)
from sklearn.metrics import r2_score
r_score=r2_score(dependent,grid_predictions)
print("The R_score value for best parameter {}:".format(grid.best_params_),r_score)


The R_score value for best parameter {'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 10}: 0.8738504648208618


In [16]:
table=pd.DataFrame.from_dict(re)

In [17]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.062819,0.033393,0.012515,0.005805,0.01,3,10,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",0.156896,0.147805,0.148109,0.155250,0.152222,0.152056,0.003669,17
1,0.088352,0.011609,0.008745,0.001050,0.01,3,100,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",0.760272,0.708444,0.738133,0.736334,0.744497,0.737536,0.016814,12
2,0.039851,0.006238,0.010017,0.002644,0.01,6,10,"{'learning_rate': 0.01, 'max_depth': 6, 'n_est...",0.156117,0.148994,0.149096,0.156087,0.152770,0.152613,0.003157,16
3,0.258000,0.060083,0.010577,0.002197,0.01,6,100,"{'learning_rate': 0.01, 'max_depth': 6, 'n_est...",0.754480,0.708375,0.740249,0.734452,0.738266,0.735164,0.015012,13
4,0.087286,0.043338,0.011399,0.008210,0.01,10,10,"{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",0.153993,0.149491,0.146381,0.156426,0.152132,0.151685,0.003491,18
5,0.536212,0.040090,0.012655,0.003805,0.01,10,100,"{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",0.736623,0.696153,0.718243,0.728861,0.719601,0.719896,0.013614,15
6,0.021114,0.001285,0.008385,0.004666,0.10,3,10,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",0.771089,0.718488,0.749434,0.746667,0.754705,0.748077,0.017046,10
7,0.086707,0.025253,0.014282,0.003837,0.10,3,100,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",0.877827,0.810826,0.891714,0.848998,0.862317,0.858337,0.027764,2
8,0.037360,0.009481,0.010402,0.006506,0.10,6,10,"{'learning_rate': 0.1, 'max_depth': 6, 'n_esti...",0.767967,0.717526,0.751604,0.745283,0.748974,0.746271,0.016332,11
9,0.153917,0.002431,0.009721,0.005576,0.10,6,100,"{'learning_rate': 0.1, 'max_depth': 6, 'n_esti...",0.849589,0.784952,0.864973,0.828706,0.849208,0.835486,0.027772,5


In [18]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 25
BMI: 25
Children: 0
Sex Male 0 or 1: 0
Smoker Yes 0 or 1: 0


In [20]:
Insurance_charges=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print("Insurance Charges={}".format(Insurance_charges))


Insurance Charges=[4878.928]
